In [1]:
import sys,os
sys.path.append(os.path.expanduser('~/smodels'))
import time
from smodels.theory.exceptions import SModelSTheoryError as SModelSError
from smodels.tools.smodelsLogging import logger
import matplotlib.pyplot as plt
from smodels.theory import element, topology
from smodels.theory.branch import Branch, decayBranches
from smodels.tools.physicsUnits import fb, GeV
from smodels.tools.smodelsLogging import logger
from smodels.tools.physicsUnits import fb,GeV,TeV
from smodels.particlesLoader import BSMList
from smodels.share.models.SMparticles import SMList
from smodels.theory.model import Model
%load_ext line_profiler

### Decomposer

In [2]:
def decompose(model, sigmacut= 0.1*fb, doCompress=True, doInvisible=True,
              minmassgap= 0*GeV):
    """
    Perform decomposition using the information stored in model.
    
    :param sigmacut: minimum sigma*BR to be generated, by default sigmacut = 0.1 fb
    :param doCompress: turn mass compression on/off
    :param doInvisible: turn invisible compression on/off
    :param minmassgap: maximum value (in GeV) for considering two R-odd particles
                       degenerate (only revelant for doCompress=True )
    :returns: list of topologies (TopologyList object)

    """
    t1 = time.time()
    
    xSectionList = model.xsections    
    pdgList = model.getValuesFor('pdg')

    if doCompress and minmassgap/GeV < 0.:
        logger.error("Asked for compression without specifying minmassgap. Please set minmassgap.")        
        raise SModelSError()

    if isinstance(sigmacut,(float,int)):
        sigmacut = sigmacut*fb
    sigmacut = sigmacut.asNumber(fb)

    xSectionList.removeLowerOrder()
    # Order xsections by PDGs to improve performance
    xSectionList.order()

    # Get maximum cross sections (weights) for single particles (irrespective
    # of sqrtS)
    maxWeight = {}
    for pid in xSectionList.getPIDs():
        maxWeight[pid] = xSectionList.getXsecsFor(pid).getMaxXsec().asNumber(fb)

    # Generate dictionary, where keys are the PIDs and values 
    # are the list of cross sections for the PID pair (for performance)
    xSectionListDict = {}    
    for pids in xSectionList.getPIDpairs():
        xSectionListDict[pids] = xSectionList.getXsecsFor(pids)

    # Create 1-particle branches with all possible mothers    
    branchList = []
    for pid in maxWeight:
        branchList.append(Branch())
        bsmParticle = model.getParticlesWith(pdg=pid)
        if not bsmParticle:
            raise SModelSError("Particle for pdg %i has not been defined.")
        if len(bsmParticle) != 1:
            raise SModelSError("Particle with pdg %i has multiple definitions.")
        branchList[-1].oddParticles = [bsmParticle[0]]
        if not pid in pdgList:
            logger.error("PDG %i has not been defined" %int(pid))
        branchList[-1].maxWeight = maxWeight[pid]  
#     print('Primary mothers = %i' %len(branchList))
        
    # Generate final branches (after all R-odd particles have decayed)
    finalBranchList = decayBranches(branchList, sigmacut)
    
#     print('Final branch list = %i' %len(finalBranchList))
    
    # Generate dictionary, where keys are the PIDs and values are the list of branches for the PID (for performance)
    branchListDict = {}
    for branch in finalBranchList:
        if branch.oddParticles[0].pdg in branchListDict:
            branchListDict[branch.oddParticles[0].pdg].append(branch)
        else:
            branchListDict[branch.oddParticles[0].pdg] = [branch]

    for pid in xSectionList.getPIDs():
        if not pid in branchListDict:
            branchListDict[pid] = []

    #Sort the branch lists by max weight to improve performance:
    for pid in branchListDict:
        branchListDict[pid] = sorted(branchListDict[pid], 
                                     key=lambda br: br.maxWeight, reverse=True)        

    smsTopList = topology.TopologyList()

    nEltot = 0
    # Combine pairs of branches into elements according to production
    # cross section list
    for pids in xSectionList.getPIDpairs():
        weightList = xSectionListDict[pids]
        maxxsec = weightList.getMaxXsec().asNumber(fb)
        if maxxsec == 0.: ## protection
            continue
        minBR = sigmacut/maxxsec
        if minBR > 1.:
            continue
        for branch1 in branchListDict[pids[0]]:
            BR1 =  branch1.maxWeight/maxWeight[pids[0]]  #Branching ratio for first branch            
            if BR1 < minBR:
                break #Stop loop if BR1 is already too low
            for branch2 in branchListDict[pids[1]]:
                BR2 =  branch2.maxWeight/maxWeight[pids[1]]  #Branching ratio for second branch                
                if BR2 < minBR:
                    break #Stop loop if BR2 is already too low        
                                     
                finalBR = BR1*BR2
                if finalBR < minBR:
                    continue # Skip elements with xsec below sigmacut
                                       
                nEltot += 1
                newElement = element.Element([branch1, branch2])
                newElement.weight = weightList*finalBR
                newElement.sortBranches()  #Make sure elements are sorted BEFORE adding them              
                smsTopList.addElement(newElement)                                                 

#     print('total number of elements = %i' %nEltot)
    smsTopList.compressElements(doCompress, doInvisible, minmassgap)
    smsTopList._setElementIds()       
#     print('total number of unique elements = %i' %len(smsTopList.getElements()))
            
#     print("decomposer done in %.2f s." % (time.time() -t1 ) )
  
    return smsTopList

### Load model

In [3]:
slhafile = '../inputFiles/slha/lightEWinos.slha'
# slhafile = '../inputFiles/slha/simplyGluino.slha'
model = Model(BSMparticles=BSMList, SMparticles=SMList)
model.updateParticles(inputFile=slhafile)


### Check with large sigmacut

In [4]:
sigmacut = 10*fb
t0 = time.time()
topList = decompose(model, sigmacut= sigmacut, doCompress=False, doInvisible=False)
print("decomposer done in %.2f s." % (time.time() -t0 ) )
print('%i unique elements' %(len(topList.getElements())))

decomposer done in 0.11 s.
103 unique elements


In [5]:
topSummary = []
for top in topList:
    total = 0.0*fb
    for el in top.getElements():
        total += el.weight.getMaxXsec()
    print(top,len(top.getElements()),total)        
    topSummary.append([len(top.getElements()),round(total.asNumber(fb),3)])
print(topSummary)

[][2] 2 1.08E-01 [pb]
[2][2] 58 6.13E+00 [pb]
[2][1,1] 1 3.19E-02 [pb]
[2][2,2] 7 4.91E-01 [pb]
[1,1][2,2] 4 8.11E-02 [pb]
[2,2][2,2] 27 1.24E+00 [pb]
[2][1,1,2] 4 8.54E-02 [pb]
[[2, 107.528], [58, 6126.453], [1, 31.861], [7, 490.94], [4, 81.12], [27, 1237.813], [4, 85.382]]


### Check with small sigmacut

In [6]:
sigmacut = 0.1*fb
t0 = time.time()
topList = decompose(model, sigmacut= sigmacut, doCompress=False, doInvisible=False)
print("decomposer done in %.2f s." % (time.time() -t0 ) )
print('%i unique elements' %(len(topList.getElements())))

decomposer done in 5.22 s.
5435 unique elements


In [7]:
topSummary = []
for top in topList:
    total = 0.0*fb
    for el in top.getElements():
        total += el.weight.getMaxXsec()
#     print(top,len(top.getElements()),total)        
    topSummary.append([len(top.getElements()),round(total.asNumber(fb),3)])

In [8]:
print(sorted(topSummary))

[[1, 3.883], [2, 1.003], [3, 1.094], [4, 0.892], [4, 2.616], [6, 2.853], [8, 3.643], [10, 3.376], [10, 14.988], [11, 10.526], [13, 13.994], [14, 11.171], [15, 164.364], [16, 78.045], [17, 30.516], [25, 12.91], [28, 19.106], [49, 42.873], [56, 25.215], [62, 48.534], [78, 51.703], [87, 150.926], [90, 6350.441], [102, 154.363], [104, 94.307], [104, 153.527], [107, 404.754], [113, 403.824], [126, 47.333], [132, 1078.776], [144, 267.975], [183, 106.275], [295, 166.074], [304, 377.261], [411, 157.421], [504, 2817.347], [648, 455.245], [747, 2086.069], [802, 1330.765]]


### Check with invisible compression

In [9]:
sigmacut = 1*fb
t0 = time.time()
topList = decompose(model, sigmacut= sigmacut, doCompress=False, doInvisible=True)
print("decomposer done in %.2f s." % (time.time() -t0 ) )
print('%i unique elements' %(len(topList.getElements())))

decomposer done in 0.96 s.
917 unique elements


### Check with mass compression

In [10]:
sigmacut = 1*fb
t0 = time.time()
topDict = decompose(model, sigmacut= sigmacut, doCompress=True, doInvisible=False)
print("decomposer done in %.2f s." % (time.time() -t0 ) )
print('%i unique elements' %(len(topDict.getElements())))

decomposer done in 0.78 s.
899 unique elements


#### Test compressed model

In [11]:
slhafile="../inputFiles/slha/higgsinoStop.slha"
model = Model(BSMList,SMList)
model.updateParticles(inputFile=slhafile,promptWidth = 1e-12*GeV)

In [12]:
sigmacut = 1*fb
t0 = time.time()
topList = decompose(model, sigmacut= sigmacut, doCompress=True, doInvisible=False, minmassgap=5*GeV)
print("decomposer done in %.2f s." % (time.time() -t0 ) )
print('%i unique elements' %(len(topList.getElements())))

decomposer done in 0.47 s.
489 unique elements


In [13]:
topSummary = []
for top in topList:
    total = 0.0*fb
    for el in top.getElements():
        total += el.weight.getMaxXsec()
#     print(top,len(top.getElements()),total)        
    topSummary.append([len(top.getElements()),round(total.asNumber(fb),3)])
print(topSummary)

[[1, 46932.601], [1, 1043.828], [14, 68354.626], [4, 8885.261], [8, 694.931], [72, 42401.477], [36, 438.84], [4, 119.091], [34, 8107.982], [116, 792.104], [16, 111.128], [111, 8119.626], [8, 23.764], [16, 47.891], [16, 41.485], [8, 11.09], [8, 11.09], [16, 19.359]]


### Test mass AND invisible compression

In [14]:
sigmacut = 0.1*fb
t0 = time.time()
topList = decompose(model, sigmacut= sigmacut, doCompress=True, doInvisible=True, minmassgap=5*GeV)
print("decomposer done in %.2f s." % (time.time() -t0 ) )
print('%i unique elements' %(len(topList.getElements())))

decomposer done in 1.91 s.
1452 unique elements


In [15]:
topSummary = []
for top in topList:
    total = 0.0*fb
    for el in top.getElements():
        total += el.weight.getMaxXsec()
#     print(top,len(top.getElements()),total)        
    topSummary.append([len(top.getElements()),round(total.asNumber(fb),3)])
print(topSummary)

[[2, 56785.133], [9, 1055.527], [22, 74853.168], [9, 10118.398], [29, 702.603], [72, 42401.477], [2, 0.923], [44, 443.058], [6, 126.609], [54, 9124.952], [48, 10.39], [284, 879.227], [17, 5.406], [22, 114.309], [120, 8125.92], [4, 1.076], [72, 48.575], [56, 99.821], [16, 2.469], [240, 119.334], [64, 31.64], [16, 1.667], [4, 0.684], [64, 31.64], [176, 59.448]]
